In [58]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from inference import get_model
import supervision as sv
from pathlib import Path
import os
from retinaface import RetinaFace

# dataset images directory
root_dir = Path("/home/rahul/chalk_following_toy_neural_network_training/dataset/images")

In [2]:
def plt_show(image: np.ndarray, figsize = (5,5)):

  plt.figure(figsize=figsize)
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.axis("off")
  plt.show()

# Blur using Supervision

This section uses Supervision to blur faces using YoloV8 model

In [2]:
root_dir = Path("/home/rahul/chalk_following_toy_neural_network_training/dataset/images")
model = get_model(model_id="yolov8n-640")

In [3]:
def pixelate_image(detection_class, image):
  pixel_annotator = sv.PixelateAnnotator()
  annotated_image = pixel_annotator.annotate(
      scene = image.copy(),
      detections=detection_class
  )
  return annotated_image

In [38]:
for image_path in root_dir.rglob("*.png"):

  image = cv2.imread(str(image_path))
  result = model.infer(image)[0]
  detection = sv.Detections.from_inference(result)
  annotated_img = pixelate_image(detection, image)
  plt_show(annotated_img)

Output hidden; open in https://colab.research.google.com to view.

# Blur using retina face

This section uses [retinaface](https://github.com/serengil/retinaface) to blur faces

In [57]:
def pixelate_face(image:np.ndarray, blocks:int=10) ->np.ndarray:

  """Taken from here: https://github.com/h0ssn1/blur-face/tree/main"""
  # divide the input image into NxN blocks
  (h, w) = image.shape[:2]
  xSteps = np.linspace(0, w, blocks + 1, dtype="int")
  ySteps = np.linspace(0, h, blocks + 1, dtype="int")
  # loop over the blocks in both the x and y direction
  for i in range(1, len(ySteps)):
      for j in range(1, len(xSteps)):
          # compute the starting and ending (x, y)-coordinates
          # for the current block
          startX = xSteps[j - 1]
          startY = ySteps[i - 1]
          endX = xSteps[j]
          endY = ySteps[i]
          # extract the ROI using NumPy array slicing, compute the
          # mean of the ROI, and then draw a rectangle with the
          # mean RGB values over the ROI in the original image
          roi = image[startY:endY, startX:endX]
          (B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
          cv2.rectangle(image, (startX, startY), (endX, endY),
                        (B, G, R), -1)
  # return the pixelated blurred image
  return image

def blur(img: np.ndarray, k: int) -> np.ndarray:
    """Taken from here: https://github.com/h0ssn1/blur-face/tree/main"""
    h, w = img.shape[:2]
    print(f"img shape: {img.shape}")
    kh, kw = h // k, w // k
    if kh % 2 == 0:
      kh -= 1
    if kw % 2 == 0:
      kw -= 1
    print(f"kh: {kh}, kw: {kw}")
    img = cv2.GaussianBlur(img, ksize=(kh, kw), sigmaX=0)
    return img

def blur_face(image: np.ndarray) -> np.ndarray:
  """Uses the above 2 function & retinaface to blur faces"""
  faces = RetinaFace.detect_faces(image)

  if not faces:
    return image
  else:
    for face in faces.values():
        x1,y1,x2,y2 = face['facial_area']

        image[y1:y2, x1:x2] = pixelate_face(blur(image[y1:y2, x1:x2], 2))
    return image

In [ ]:
result_dir = root_dir.parent / Path("blured_images")
result_dir.mkdir(exist_ok=True)
for image_path in root_dir.rglob("*.png"):

    print(f" img_path: {image_path}")
    image = cv2.imread(str(image_path))
    image = blur_face(image)
    ret = cv2.imwrite(
        str(
            result_dir/ image_path.name
        ),
        image
    )
    if ret == False: raise FileExistsError
    plt_show(image)